In [1]:
%matplotlib inline

In [2]:
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt
import seaborn as sns
from nighres.io import io_mesh
import gdist
from brainsmash.mapgen.base import Base
from brainsmash.mapgen.sampled import Sampled
from brainsmash.mapgen.memmap import txt2memmap

In [3]:
data_dir = '/home/julia/data/gradients/'

### Create Geodesic distance map inside cortex mask

In [16]:
mesh = io_mesh.load_mesh(data_dir+'results/embedding/embed_sampled_mesh.vtk')
cortex = np.where(mesh['data'][:,0]!=0)[0]
np.save(data_dir+'results/null_models/surface/cortex_mask.npy', cortex)
np.save('/home/julia/data/gradients/results/null_models/surface/points.npy', mesh['points'])
np.save('/home/julia/data/gradients/results/null_models/surface/faces.npy', mesh['faces'])

In [6]:
cortex = np.load(data_dir+'results/null_models/surface/cortex_mask.npy')
points = np.load(data_dir+'results/null_models/surface/points.npy')
faces = np.load(data_dir+'results/null_models/surface/faces.npy')

In [7]:
dist_matrix = np.zeros((500, cortex.shape[0]))

In [ ]:
for c in range(500):
    dist = gdist.compute_gdist(np.array(points, dtype=np.float64), 
                               np.array(faces, dtype=np.int32), 
                               source_indices=np.array([cortex[c+300]], dtype=np.int32),
                               target_indices=np.array(cortex, dtype=np.int32))
    dist[dist==np.inf] = 0
    dist_matrix[c,:] = dist

In [ ]:
np.savetxt(data_dir + 'results/null_models/surface/cortex_geodesic_300_799.txt', dist_matrix)

In [ ]:
dist = gdist.local_gdist_matrix(np.array(mesh['points'], dtype=np.float64), 
                                np.array(mesh['faces'], dtype=np.int32))

In [ ]:
## Construct full matrix
dist_full = np.zeros(shape=(mesh['points'].shape[0], mesh['points'].shape[0]))
dist_full[np.triu_indices(mesh['points'].shape[0], k=1)] = dist
dist_full += dist_full.T
# Write to disk
np.savetxt(data_dir + 'results/null_models/surface/cortex_geodesic.txt', dist_full)

In [ ]:
output_files = txt2memmap(data_dir+"results/null_models/cortex_geodesic.txt", 
                          data_dir+"results/null_models", 
                          maskfile=None, delimiter=' ')